In [1]:
def Preprocess(file):
    array = []
    for line in f:
        array.append(line)
    xc = [0]*300
    yc = [0]*300
    maxrows = 300
    maxcols = 300
    Cost = [([0] * maxcols) for row in range(maxrows) ]

    dim = int(array[0])
    for it in range(1, dim+1):
        p = array[it]
        tokens = p.split()
        xc[it-1] = float(tokens[0])
        yc[it-1] = float(tokens[1])
    
    for i in range(0, dim):
        for j in range(0, dim):
            Cost[i][j] = pow((pow(xc[i]-xc[j], 2) + pow(yc[i] - yc[j], 2)), 0.5)
        
    for i in range(0, dim):
        for j in range(0, dim):
            if(i==j):
                Cost[i][j] = 999
    
    Cost_No1 = [([0] * (dim-1)) for row in range(maxrows-1) ]
    for i in range(0, dim-1):
        for j in range(0, dim-1):
            Cost_No1[i][j] = Cost[i+1][j+1] 
    for i in range(0, dim-1):
        for j in range(0, dim-1):
            if (i == j):
                Cost_No1[i][j] = 0
    
    return Cost, xc, yc, dim

In [2]:
def PrimMST(Adj, V):
    newdim = len(V)
    ## Output MST is a list of edges 
    MST = {}
    ## dist[v] is the distance of the vertex v from the parent of v in the MST
    dist = [sys.maxsize]*newdim
    dist[0] = 0
    ## parent[v] is the parent of vertex v in the MST
    parent = [None]*newdim 
    parent[0] = -1
    ## mstset{v} indicates if a vertex is included in the mst or not initialize with False 
    mstset = [False]*newdim 

    
    for l in range(newdim):
        v1 = Extractmin(dist, mstset, newdim)
        mstset[v1] = True
        for v2 in range(newdim):
            if(Adj[v1][v2] > 0 and mstset[v2] == False and dist[v2] > Adj[v1][v2]):
                dist[v2] = Adj[v1][v2]
                parent[v2] = v1

    j = 0
    for v in range(newdim):
        if(parent[v] != -1): ## parent of 0 is -1
            edge = (parent[v], v)
            costedge = Adj[parent[v]][v]
            MST[edge] = costedge
    return MST


            
### find the vertex with the min dist value not included in the MST             
def Extractmin(dist, mstset, dim):
    m = sys.maxsize
    for v in range(dim):
        if(dist[v] < m and mstset[v] == False):
            m = dist[v]
            minin = v
    return minin
    
            

In [3]:

def CreateOnetree(Cost, dim):
    Onetree = {}
    Edgecost0 = [0]*dim
    for i in range(0, dim):
        Edgecost0[i] = Cost[0][i]
    Edgecost = sorted(Edgecost0)

    minedge_1 = Edgecost[0]
    minedge_2 = Edgecost[1]

    for i in range(0, dim):
        if(Edgecost0[i] == minedge_1):
            vertex1 = i
            break
    for j in range(0, dim):
        if(Edgecost0[j] == minedge_2 and j!= vertex1):
            vertex2 = j
    e1 = (0, vertex1)
    e2 = (0, vertex2)
    cost1 = Cost[0][vertex1]
    cost2 = Cost[0][vertex2]
    Onetree[e1] = cost1
    Onetree[e2] = cost2
    ##### remove root node and create the MST of the remaining nodes ####
    V_no1 = []
    for i in range(dim-1):
        V_no1.append(i)
    Cost_No1 = [([0] * (dim-1)) for row in range(dim-1) ]
    for i in range(0, dim-1):
        for j in range(0, dim-1):
            Cost_No1[i][j] = Cost[i+1][j+1] 
    for i in range(0, dim-1):
        for j in range(0, dim-1):
            if (i == j):
                Cost_No1[i][j] = 0        
    Adj = Cost_No1
    MST = PrimMST(Adj, V_no1)
    ## Actually in the MST call vertex 0 was not included. So, the actual MST vertex numbers are 1 more
    actualmst = {}
    for e in MST:
        v1 = e[0]
        v2 = e[1]
        newedge = (v1+1, v2+1)
        actualmst[newedge] = MST[e]
    ##### Onetree has all the edges of actualmst ####
    for e in actualmst:
        Onetree[e] = actualmst[e]
    return Onetree



In [4]:
def Subgradient1(Cost, dim, maxi):
    K = 500
    u = [0]*dim
    d = [0]*dim
    M = 10
    rho = 0.7
    flag = False
    ite = 0
    tempcost = [([0] * dim) for row in range(dim) ]
    for i in range(0, dim):
        for j in range(0, dim):
            tempcost[i][j] = Cost[i][j]
            
    while(flag == False):
        flag = True
        onetree = CreateOnetree(Cost, dim)
        Cost_Onetree = 0
        for edge in onetree:
             Cost_Onetree =  Cost_Onetree + onetree[edge]
        ###### update d ########
        sum_xe = [0]*dim
        for v in range(0, dim):
            for edge in onetree:
                if(edge[0] == v or edge[1] == v):
                    sum_xe[v] = sum_xe[v] +1
        for v in range(0, dim):
            d[v] = 2 - sum_xe[v] 
        ######### update u #######
        temp = u
        mu = M*pow(rho, ite)
        for v in range(0, dim):
            u[v] = temp[v] + mu*d[v]
        ############# update cost #######
        for i in range(0, dim):
            for j in range(0, dim):
                Cost[i][j] = tempcost[i][j] - u[i] - u[j]
            #Cost[j][i] = Cost[j][i] - u[i] - u[j]
        ite = ite+1
        for i in range(0, dim):
            if(d[i] != 0 and ite < maxi):
                flag = False

    sumu = 0
    for i in range(dim):
        sumu = sumu + u[i]
    Objective = Cost_Onetree + sumu
    return(Objective)

In [7]:
from gurobipy import *
import string
import time
import sys

f = open("C:\\Users\\Prasangsha\\Downloads\\TSP_instance_n_5_s_1.dat")
Cost, xc, yc, dim = Preprocess(f)

maxiteration = 1000
Objective = Subgradient1(Cost, dim, maxiteration)
print('\n Best Objective found:', Objective)


[[999, 13.25004612393162, 10.605055562620615, 3.478730796099461, 2.952790992999422, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13.25004612393162, 999, 16.50

[[999.0, 7.12104612393162, 8.634055562620615, 7.507730796099461, 7.023790992999423, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [7.12104612393162, 986.742, 8.